## LangChain 라이브러리를 활용한 RAG 구성

### 필수 라이브러리 설치

In [ ]:
pip install langchain
pip install langchain-openai
pip install tiktoken
pip install langchain_community

### OpenAI 인증키 등록

In [10]:
# OpenAI 인증키 등록
import os
os.environ['OPENAI_API_KEY'] = ''

### 기본 LLM Chain 생성

In [11]:
from langchain_openai import ChatOpenAI

# model
llm = ChatOpenAI(model='gpt-4o')
# chain 실행
llm.invoke("LangChain Test, Udap LLM Assistant와 잘 연결 되었나")


AIMessage(content='네, 잘 연결되었습니다. LangChain과 Udap LLM Assistant를 통해 질문이나 요청이 있다면 도와드리겠습니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 21, 'total_tokens': 57, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_45cf54deae', 'finish_reason': 'stop', 'logprobs': None}, id='run-f4f32613-ecd1-4a36-92f9-f193a2dae98f-0', usage_metadata={'input_tokens': 21, 'output_tokens': 36, 'total_tokens': 57, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### LLM Template 적용

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# instruction이 들어가는 부분
assistantInstuc = 'You are a query generator for a service called Udap QM. <Question>: {input}'

# 사용자 질문 변수
userQues = 'hello, who are you?'

# 템플릿 생성
prompt = ChatPromptTemplate.from_template(assistantInstuc)
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are a query generator for a service called Udap QM. <Question>: {input}'), additional_kwargs={})])

In [ ]:
# 템플릿과 llm chain 생성
testLChain = prompt | llm
testLChain.invoke({"input": userQues})

AIMessage(content='Hello! How can I assist you today? If you have a specific topic or question in mind related to Udap QM, feel free to let me know so I can generate the appropriate query for you.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 24, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_159d8341cc', 'finish_reason': 'stop', 'logprobs': None}, id='run-28b75fc2-6a44-49b4-863a-046f79ffb537-0', usage_metadata={'input_tokens': 24, 'output_tokens': 41, 'total_tokens': 65, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### LangChain의 Runnable 프로토콜

- **invoke**: 주어진 입력에 대해 체인을 호출하고, 결과를 반환합니다. 이 메소드는 단일 입력에 대해 동기적으로 작동합니다.
- **batch**: 입력 리스트에 대해 체인을 호출하고, 각 입력에 대한 결과를 리스트로 반환합니다. 이 메소드는 여러 입력에 대해 동기적으로 작동하며, 효율적인 배치 처리를 가능하게 합니다.
- **stream**: 입력에 대해 체인을 호출하고, 결과의 조각들을 스트리밍합니다. 이는 대용량 데이터 처리나 실시간 데이터 처리에 유용합니다.
- **비동기 버전**: ainvoke, abatch, astream 등의 메소드는 각각의 동기 버전에 대한 비동기 실행을 지원합니다. 이를 통해 비동기 프로그래밍 패러다임을 사용하여 더 높은 처리 성능과 효율을 달성할 수 있습니다.

In [ ]:
from langchain.schema.output_parser import StrOutputParser

output_parser = StrOutputParser()
testLChain = prompt | llm | output_parser

# batch 메소드
userTopic = ["who", "what", "how"]
results = testLChain.batch([{"topic": t} for t in userTopic])
for topic, result in zip(userTopic, results):
    print(f"{topic}: {result[:10]}...")

### LLM 모델 파라미터 설정 (Udap LLM 기준 조정 필요)

- Temperature: 생성된 텍스트의 다양성을 조정합니다. 값이 작으면 예측 가능하고 일관된 출력을 생성하는 반면, 값이 크면 다양하고 예측하기 어려운 출력을 생성합니다.

   > *낮은 값 (0.0~0.3). 정확하고 일관된 쿼리를 생성하려면 높은 다양성보다는 예측 가능성을 우선시해야 함*

- Max Tokens (최대 토큰 수): 생성할 최대 토큰 수를 지정합니다. 생성할 텍스트의 길이를 제한합니다.

   > *최소한의 토큰으로 설정*
- Top P (Top Probability): 생성 과정에서 특정 확률 분포 내에서 상위 P% 토큰만을 고려하는 방식입니다. 이는 출력의 다양성을 조정하는 데 도움이 됩니다.

   > *낮게 설정 (0.1~0.5). 상위 확률 내에서만 선택되도록 하여, 예측 가능한 출력을 유지*
- Frequency Penalty (빈도 패널티): 값이 클수록 이미 등장한 단어나 구절이 다시 등장할 확률을 감소시킵니다. 이를 통해 반복을 줄이고 텍스트의 다양성을 증가시킬 수 있습니다. (0~1)

   > *0.2~0.5 사이의 값 쿼리 생성 시 중복을 줄이는 효과가 있어, 더 명확한 쿼리가 나올 수 있음*
- Presence Penalty (존재 패널티): 텍스트 내에서 단어의 존재 유무에 따라 그 단어의 선택 확률을 조정합니다. 값이 클수록 아직 텍스트에 등장하지 않은 새로운 단어의 사용이 장려됩니다. (0~1)

   > *SQL 쿼리에서는 새로운 단어의 사용을 장려할 필요가 크지 않으므로, 0 또는 낮은 값을 사용*
- Stop Sequences (정지 시퀀스): 특정 단어나 구절이 등장할 경우 생성을 멈추도록 설정합니다. 이는 출력을 특정 포인트에서 종료하고자 할 때 사용됩니다.

   > *; 또는 쿼리 완료를 의미하는 특정 구문*

In [17]:
model_param = {
    "temperature": 0.1,
    "max_tokens": 1000,
}

kwargs = {
    "frequency_penalty": 0,   
    "presence_penalty": 0,
    "stop": [";"]  
}

llmModel = ChatOpenAI(model='gpt-4o', **model_param, model_kwargs=kwargs)

c:\Users\khlee\anaconda3\envs\py3.9.7\lib\site-packages\IPython\core\interactiveshell.py:3338: UserWarning: Parameters {'presence_penalty', 'frequency_penalty', 'stop'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


### RAG 구성하기

1) 데이터 로드

In [28]:
from langchain_community.document_loaders import TextLoader
import os

os.chdir('C:\\Users\\khlee\\Desktop\\khlee\\13. udap23\\git\\llm')

loader = TextLoader('llm_vector.txt')
qm_schema = loader.load()

print(qm_schema)
print(len(qm_schema[0].page_content))

[Document(metadata={'source': 'llm_vector.txt'}, page_content='{\n    "Users": {\n        "columns": ["userId", "userCompany", "userConfig", "userDisable", "userEmail", "userGroup", "userName", "userPhone", "userPw", "userRole", "userSince"],\n        "relations": []\n    },\n    "Template": {\n        "columns": ["id", "eqType", "prtNote", "prtRevision", "users_userId", "majorRev"],\n        "relations": ["Users"]\n    }, \n    "Report": {\n        "columns": ["prDid", "eqSerial", "eqType", "template_id", "users_userId"],\n        "relations": ["Users", "Template"]\n    },\n    "MainCategory": {\n        "columns": ["mainId", "mainLid", "mainVal", "template_main_id", "mainSeq"],\n        "relations": ["Template"]\n    }, \n    "SubCategory": {\n        "columns": ["subId", "subLid", "subSid", "subVal", "template_sub_id", "subSeq"],\n        "relations": ["Template"]\n    }, \n    "Columns": {\n        "columns": ["colId", "colCid", "colLid", "colSid", "colVal", "template_col_id"],\n  

2) Text Splitter

- separator: 분할된 각 청크를 구분할 때 기준이 되는 문자열
- chunk_size: 각 청크의 최대 길이
- chunk_overlap: 인접한 청크 사이에 중복으로 포함될 문자의 수
- length_function: 청크의 길이

In [32]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size= 200,
    length_function = len
)

splitted_schema = text_splitter.split_text(qm_schema[0].page_content)

splitted_schema

['{\n    "Users": {\n        "columns": ["userId", "userCompany", "userConfig", "userDisable", "userEmail", "userGroup", "userName", "userPhone", "userPw", "userRole", "userSince"],',
 '"Users": {\n        "columns": ["userId", "userCompany", "userConfig", "userDisable", "userEmail", "userGroup", "userName", "userPhone", "userPw", "userRole", "userSince"],\n        "relations": []',
 '"columns": ["userId", "userCompany", "userConfig", "userDisable", "userEmail", "userGroup", "userName", "userPhone", "userPw", "userRole", "userSince"],\n        "relations": []\n    },',
 '"relations": []\n    },\n    "Template": {\n        "columns": ["id", "eqType", "prtNote", "prtRevision", "users_userId", "majorRev"],\n        "relations": ["Users"]\n    }, \n    "Report": {',
 '"relations": ["Users"]\n    }, \n    "Report": {\n        "columns": ["prDid", "eqSerial", "eqType", "template_id", "users_userId"],\n        "relations": ["Users", "Template"]\n    },',
 '}, \n    "Report": {\n        "colum